```sh
ollama pull mxbai-embed-large
ollama pull llama2
ollama pull llama3.1
```

In [7]:
import requests
url     = 'https://notre-chile-christine-affecting.trycloudflare.com'
headers = {}
payload = { 'name' : 'gemma:2b' }
res = requests.post(url + '/api/pull', json=payload, headers=headers)
payload = { 'name' : 'mxbai-embed-large' }
res = requests.post(url + '/api/pull', json=payload, headers=headers)
payload = { 'name' : 'llama2' }
res = requests.post(url + '/api/pull', json=payload, headers=headers)
payload = { 'name' : 'llama3.1' }
res = requests.post(url + '/api/pull', json=payload, headers=headers)

In [10]:
%env OLLAMA_HOST=https://notre-chile-christine-affecting.trycloudflare.com
%env OLLAMA_HOST

env: OLLAMA_HOST=https://notre-chile-christine-affecting.trycloudflare.com


'https://notre-chile-christine-affecting.trycloudflare.com'

In [12]:
%pip install ollama chromadb

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [18]:
import ollama
import chromadb



documents = [
  "Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels",
  "Llamas were first domesticated and used as pack animals 4,000 to 5,000 years ago in the Peruvian highlands",
  "Llamas can grow as much as 6 feet tall though the average llama between 5 feet 6 inches and 5 feet 9 inches tall",
  "Llamas weigh between 280 and 450 pounds and can carry 25 to 30 percent of their body weight",
  "Llamas are vegetarians and have very efficient digestive systems",
  "Llamas live to be about 20 years old, though some only live for 15 years and others live to be 30 years old",
]

client = chromadb.Client()
collections = client.list_collections()
if len(collections) > 0:
  collection = client.delete_collection(name="docs")
collection = client.create_collection(name="docs")

# store each document in a vector embedding database
for i, d in enumerate(documents):
  response = ollama.embeddings(model="mxbai-embed-large", prompt=d)
  embedding = response["embedding"]
  collection.add(
    ids=[str(i)],
    embeddings=[embedding],
    documents=[d]
  )


# an example prompt
prompt = "What animals are llamas related to?"

# generate an embedding for the prompt and retrieve the most relevant doc
response = ollama.embeddings(
  prompt=prompt,
  model="mxbai-embed-large"
)
results = collection.query(
  query_embeddings=[response["embedding"]],
  n_results=1
)
data = results['documents'][0][0]

print(data)

# generate a response combining the prompt and data we retrieved in step 2
output = ollama.generate(
  model="llama2",
  prompt=f"Using this data: {data}. Respond to this prompt: {prompt}"
)

print(output['response'])

Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels

Llamas are related to several other animals within the camelid family. Specifically, they are most closely related to vicuñas and camels. Both vicuñas and camels are also members of the camelid family, and share many physical and behavioral characteristics with llamas. These include:

* A distinctive set of teeth, with a flat upper surface and a curved lower surface
* Long eyelashes and ear hairs
* A thick, woolly coat that can be used for insulation in cold climates
* The ability to go without water for long periods of time, allowing them to survive in arid environments
* A unique digestive system that allows them to extract moisture from plants more efficiently than other mammals.

In addition to their close relationship with vicuñas and camels, llamas are also distantly related to alpacas, which are also members of the camelid family. While llamas and alpacas share some similaritie

In [19]:
%pip install langchain langchain_community langchain-openai scikit-learn langchain-ollama

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [20]:
import os
from langchain_community.llms import Ollama
from dotenv import load_dotenv
from langchain_community.embeddings import OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import create_retrieval_chain
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain

llm = Ollama(model="llama3.1", base_url="https://notre-chile-christine-affecting.trycloudflare.com")

embed_model = OllamaEmbeddings(
    model="llama3.1",
    base_url='https://notre-chile-christine-affecting.trycloudflare.com'
)

text = """
    In the lush canopy of a tropical rainforest, two mischievous monkeys, Coco and Mango, swung from branch to branch, their playful antics echoing through the trees. They were inseparable companions, sharing everything from juicy fruits to secret hideouts high above the forest floor. One day, while exploring a new part of the forest, Coco stumbled upon a beautiful orchid hidden among the foliage. Entranced by its delicate petals, Coco plucked it and presented it to Mango with a wide grin. Overwhelmed by Coco's gesture of friendship, Mango hugged Coco tightly, cherishing the bond they shared. From that day on, Coco and Mango ventured through the forest together, their friendship growing stronger with each passing adventure. As they watched the sun dip below the horizon, casting a golden glow over the treetops, they knew that no matter what challenges lay ahead, they would always have each other, and their hearts brimmed with joy.
    """

text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
chunks = text_splitter.split_text(text)

vector_store = Chroma.from_texts(chunks, embed_model)

retriever = vector_store.as_retriever()

chain = create_retrieval_chain(combine_docs_chain=llm,retriever=retriever)

retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)

retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)    

response = retrieval_chain.invoke({"input": "Tell me name of monkeys and where do they live"})
print(response['answer'])

/tmp/ipykernel_1419/86062808.py:12: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.1", base_url="https://notre-chile-christine-affecting.trycloudflare.com")
/tmp/ipykernel_1419/86062808.py:14: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embed_model = OllamaEmbeddings(


/workspace/.pyenv_mirror/user/current/lib/python3.10/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


Coco and Mango are the two mischievous monkeys, and they live in a tropical rainforest.
